<a href="https://colab.research.google.com/github/Dharanidharan457/ShadowFox/blob/main/circketfieldanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

class CricketFieldingAnalysis:
    def __init__(self):
        self.fielding_data = pd.DataFrame(columns=[
            'Match_No', 'Innings', 'Team', 'Player_Name', 'Ballcount', 'Position',
            'Short_Description', 'Pick', 'Throw', 'Runs', 'Overcount', 'Venue',
            'Date', 'CP', 'GT', 'C', 'DC', 'ST', 'RO', 'MRO', 'DH', 'RS'
        ])

        self.weights = {
            'WCP': 1,
            'WGT': 1.5,
            'WC': 2,
            'WDC': -2,
            'WST': 3,
            'WRO': 3,
            'WMRO': -1,
            'WDH': 2
        }

    def set_weights(self, new_weights):

        for key, value in new_weights.items():
            if key in self.weights:
                self.weights[key] = value

    def add_fielding_event(self, match_no, innings, team,
                           player_name, ballcount, position,
                          description, pick, throw, runs,
                           overcount, venue, date=None):

        if date is None:
            date = datetime.now().strftime('%Y-%m-%d')


        cp = 1 if pick == 'clean pick' else 0
        gt = 1 if pick == 'good throw' else 0
        c = 1 if pick == 'catch' else 0
        dc = 1 if pick == 'drop catch' else 0
        st = 1 if throw == 'stumping' else 0
        ro = 1 if throw == 'run out' else 0
        mro = 1 if throw == 'missed run out' else 0
        dh = 1 if description and 'direct hit' in description.lower() else 0
        rs = runs


        new_row = pd.DataFrame({
            'Match_No': [match_no],
            'Innings': [innings],
            'Team': [team],
            'Player_Name': [player_name],
            'Ballcount': [ballcount],
            'Position': [position],
            'Short_Description': [description],
            'Pick': [pick],
            'Throw': [throw],
            'Runs': [runs],
            'Overcount': [overcount],
            'Venue': [venue],
            'Date': [date],
            'CP': [cp],
            'GT': [gt],
            'C': [c],
            'DC': [dc],
            'ST': [st],
            'RO': [ro],
            'MRO': [mro],
            'DH': [dh],
            'RS': [rs]
        })


        self.fielding_data = pd.concat([self.fielding_data, new_row], ignore_index=True)
        print(f"Fielding event added for {player_name} at {position}.")

    def calculate_performance_score(self, player_name=None, match_no=None):

        if self.fielding_data.empty:
            print("No fielding data available. Please add data first.")
            return pd.DataFrame()

        # Filter data if parameters are provided
        filtered_data = self.fielding_data
        if player_name:
            filtered_data = filtered_data[filtered_data['Player_Name'] == player_name]
        if match_no:
            filtered_data = filtered_data[filtered_data['Match_No'] == match_no]

        if filtered_data.empty:
            print("No data matching the specified criteria.")
            return pd.DataFrame()


        grouped = filtered_data.groupby(['Match_No', 'Player_Name']).agg({
            'CP': 'sum',
            'GT': 'sum',
            'C': 'sum',
            'DC': 'sum',
            'ST': 'sum',
            'RO': 'sum',
            'MRO': 'sum',
            'DH': 'sum',
            'RS': 'sum',
            'Team': 'first'
        }).reset_index()


        grouped['Performance_Score'] = (
            grouped['CP'] * self.weights['WCP'] +
            grouped['GT'] * self.weights['WGT'] +
            grouped['C'] * self.weights['WC'] +
            grouped['DC'] * self.weights['WDC'] +
            grouped['ST'] * self.weights['WST'] +
            grouped['RO'] * self.weights['WRO'] +
            grouped['MRO'] * self.weights['WMRO'] +
            grouped['DH'] * self.weights['WDH'] +
            grouped['RS']
        )

        return grouped.sort_values(by='Performance_Score', ascending=False)

    def export_data(self, filename='cricket_fielding_data.csv'):
        if self.fielding_data.empty:
            print("No data to export.")
            return

        self.fielding_data.to_csv(filename, index=False)
        print(f"Data exported to {filename}")

    def import_data(self, filename):

        try:
            imported_data = pd.read_csv(filename)

            required_cols = ['Match_No', 'Innings', 'Team', 'Player_Name', 'Position', 'Pick', 'Throw', 'Runs']
            if not all(col in imported_data.columns for col in required_cols):
                print("The imported file does not have all required columns.")
                return


            indicator_cols = ['CP', 'GT', 'C', 'DC', 'ST', 'RO', 'MRO', 'DH', 'RS']
            for col in indicator_cols:
                if col not in imported_data.columns:
                    imported_data[col] = 0


            if 'CP' in imported_data.columns and imported_data['CP'].sum() == 0:
                imported_data['CP'] = imported_data['Pick'].apply(lambda x: 1 if x == 'clean pick' else 0)
                imported_data['GT'] = imported_data['Pick'].apply(lambda x: 1 if x == 'good throw' else 0)
                imported_data['C'] = imported_data['Pick'].apply(lambda x: 1 if x == 'catch' else 0)
                imported_data['DC'] = imported_data['Pick'].apply(lambda x: 1 if x == 'drop catch' else 0)
                imported_data['ST'] = imported_data['Throw'].apply(lambda x: 1 if x == 'stumping' else 0)
                imported_data['RO'] = imported_data['Throw'].apply(lambda x: 1 if x == 'run out' else 0)
                imported_data['MRO'] = imported_data['Throw'].apply(lambda x: 1 if x == 'missed run out' else 0)
                imported_data['DH'] = imported_data['Short_Description'].apply(
                    lambda x: 1 if isinstance(x, str) and 'direct hit' in x.lower() else 0
                )
                imported_data['RS'] = imported_data['Runs']

            self.fielding_data = imported_data
            print(f"Successfully imported data from {filename}")
        except Exception as e:
            print(f"Error importing data: {e}")

    def player_summary(self, player_name=None, match_no=None):

        if self.fielding_data.empty:
            print("No fielding data available.")
            return

        # Get performance scores
        performance = self.calculate_performance_score(player_name, match_no)

        if performance.empty:
            print("No performance data available for the specified criteria.")
            return

        # Print summary
        print("\n===== FIELDING PERFORMANCE SUMMARY =====")
        for _, row in performance.iterrows():
            print(f"\nPlayer: {row['Player_Name']} | Team: {row['Team']} | Match: {row['Match_No']}")
            print(f"Clean Picks: {row['CP']}")
            print(f"Good Throws: {row['GT']}")
            print(f"Catches: {row['C']}")
            print(f"Dropped Catches: {row['DC']}")
            print(f"Stumpings: {row['ST']}")
            print(f"Run Outs: {row['RO']}")
            print(f"Missed Run Outs: {row['MRO']}")
            print(f"Direct Hits: {row['DH']}")
            print(f"Runs Saved: {row['RS']}")
            print(f"Performance Score: {row['Performance_Score']:.2f}")
            print("-" * 40)

    def team_analysis(self, team_name=None, match_no=None):

        if self.fielding_data.empty:
            print("No fielding data available.")
            return


        filtered_data = self.fielding_data
        if team_name:
            filtered_data = filtered_data[filtered_data['Team'] == team_name]
        if match_no:
            filtered_data = filtered_data[filtered_data['Match_No'] == match_no]

        if filtered_data.empty:
            print("No data matching the specified criteria.")
            return


        team_summary = filtered_data.groupby(['Match_No', 'Team']).agg({
            'CP': 'sum',
            'GT': 'sum',
            'C': 'sum',
            'DC': 'sum',
            'ST': 'sum',
            'RO': 'sum',
            'MRO': 'sum',
            'DH': 'sum',
            'RS': 'sum'
        }).reset_index()


        print("\n===== TEAM FIELDING SUMMARY =====")
        for _, row in team_summary.iterrows():
            print(f"\nTeam: {row['Team']} | Match: {row['Match_No']}")
            print(f"Total Clean Picks: {row['CP']}")
            print(f"Total Good Throws: {row['GT']}")
            print(f"Total Catches: {row['C']}")
            print(f"Total Dropped Catches: {row['DC']}")
            print(f"Total Stumpings: {row['ST']}")
            print(f"Total Run Outs: {row['RO']}")
            print(f"Total Missed Run Outs: {row['MRO']}")
            print(f"Total Direct Hits: {row['DH']}")
            print(f"Total Runs Saved: {row['RS']}")


            if row['C'] + row['DC'] > 0:
                catch_efficiency = (row['C'] / (row['C'] + row['DC'])) * 100
                print(f"Catch Efficiency: {catch_efficiency:.2f}%")


            if row['RO'] + row['MRO'] > 0:
                run_out_efficiency = (row['RO'] / (row['RO'] + row['MRO'])) * 100
                print(f"Run Out Efficiency: {run_out_efficiency:.2f}%")

            print("-" * 40)

    def visualize_player_performance(self, match_no=None, top_n=5):

        if self.fielding_data.empty:
            print("No fielding data available.")
            return


        performance = self.calculate_performance_score(match_no=match_no)

        if performance.empty:
            print("No performance data available for visualization.")
            return


        performance['Performance_Score'] = performance['Performance_Score'].astype(float)
        top_players = performance.nlargest(top_n, 'Performance_Score')


        plt.figure(figsize=(12, 8))


        plt.subplot(2, 1, 1)
        sns.barplot(x='Player_Name', y='Performance_Score', hue='Team', data=top_players)
        plt.title(f'Top {top_n} Players Performance Scores')
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Stacked bar
        plt.subplot(2, 1, 2)
        action_data = top_players[['Player_Name', 'CP', 'GT', 'C', 'RO', 'ST', 'DH']].set_index('Player_Name')
        action_data.plot(kind='bar', stacked=True, figsize=(12, 6))
        plt.title('Fielding Actions Breakdown')
        plt.xticks(rotation=45)
        plt.tight_layout()

        plt.show()

    def position_analysis(self, match_no=None):
        """Analyze fielding performance by position."""
        if self.fielding_data.empty:
            print("No fielding data available.")
            return

        # Filter data if match is specified
        filtered_data = self.fielding_data
        if match_no:
            filtered_data = filtered_data[filtered_data['Match_No'] == match_no]

        if filtered_data.empty:
            print("No data matching the specified criteria.")
            return

        # Group by position
        position_data = filtered_data.groupby('Position').agg({
            'CP': 'sum',
            'GT': 'sum',
            'C': 'sum',
            'DC': 'sum',
            'ST': 'sum',
            'RO': 'sum',
            'MRO': 'sum',
            'DH': 'sum',
            'RS': 'sum'
        }).reset_index()

        # Calculate total actions per position
        position_data['Total_Actions'] = position_data['CP'] + position_data['GT'] + position_data['C'] + \
                                        position_data['ST'] + position_data['RO'] + position_data['DH']

        # Print position analysis
        print("\n===== FIELDING POSITION ANALYSIS =====")
        for _, row in position_data.sort_values(by='Total_Actions', ascending=False).iterrows():
            print(f"\nPosition: {row['Position']}")
            print(f"Total Actions: {row['Total_Actions']}")
            print(f"Clean Picks: {row['CP']}")
            print(f"Good Throws: {row['GT']}")
            print(f"Catches: {row['C']}")
            print(f"Dropped Catches: {row['DC']}")
            print(f"Stumpings: {row['ST']}")
            print(f"Run Outs: {row['RO']}")
            print(f"Missed Run Outs: {row['MRO']}")
            print(f"Direct Hits: {row['DH']}")
            print(f"Runs Saved: {row['RS']}")

            # Calculate effectiveness
            if row['Total_Actions'] > 0:
                effectiveness = ((row['CP'] + row['GT'] + row['C'] + row['ST'] + row['RO'] + row['DH']) /
                                (row['Total_Actions'] + row['DC'] + row['MRO'])) * 100
                print(f"Position Effectiveness: {effectiveness:.2f}%")

            print("-" * 40)



def demo_with_sample_data():


    analysis = CricketFieldingAnalysis()


    analysis.add_fielding_event(
        match_no="IPL2023-045",
        innings=1,
        team="Mumbai Indians",
        player_name="Rohit Sharma",
        ballcount=2,
        position="Mid-off",
        description="Clean pick and quick throw to keeper",
        pick="clean pick",
        throw="",
        runs=1,
        overcount=1,
        venue="Wankhede Stadium"
    )

    analysis.add_fielding_event(
        match_no="IPL2023-045",
        innings=1,
        team="Mumbai Indians",
        player_name="Rohit Sharma",
        ballcount=4,
        position="Slip",
        description="Dropped a difficult catch",
        pick="drop catch",
        throw="",
        runs=-1,
        overcount=3,
        venue="Wankhede Stadium"
    )

    analysis.add_fielding_event(
        match_no="IPL2023-045",
        innings=1,
        team="Mumbai Indians",
        player_name="Rohit Sharma",
        ballcount=3,
        position="Cover",
        description="Brilliant diving catch",
        pick="catch",
        throw="",
        runs=2,
        overcount=8,
        venue="Wankhede Stadium"
    )

    # Player 2: Jasprit Bumrah
    analysis.add_fielding_event(
        match_no="IPL2023-045",
        innings=1,
        team="Mumbai Indians",
        player_name="Jasprit Bumrah",
        ballcount=1,
        position="Fine Leg",
        description="Good throw to keeper",
        pick="good throw",
        throw="",
        runs=1,
        overcount=5,
        venue="Wankhede Stadium"
    )

    analysis.add_fielding_event(
        match_no="IPL2023-045",
        innings=1,
        team="Mumbai Indians",
        player_name="Jasprit Bumrah",
        ballcount=5,
        position="Long On",
        description="Direct hit from boundary for run out",
        pick="good throw",
        throw="run out",
        runs=2,
        overcount=12,
        venue="Wankhede Stadium"
    )

    analysis.add_fielding_event(
        match_no="IPL2023-045",
        innings=1,
        team="Mumbai Indians",
        player_name="Jasprit Bumrah",
        ballcount=6,
        position="Third Man",
        description="Fumbled the ball allowing extra run",
        pick="fumble",
        throw="",
        runs=-1,
        overcount=15,
        venue="Wankhede Stadium"
    )

    # Player 3: Ishan Kishan (Wicketkeeper)
    analysis.add_fielding_event(
        match_no="IPL2023-045",
        innings=1,
        team="Mumbai Indians",
        player_name="Ishan Kishan",
        ballcount=3,
        position="Wicketkeeper",
        description="Clean collection and stumping",
        pick="clean pick",
        throw="stumping",
        runs=2,
        overcount=7,
        venue="Wankhede Stadium"
    )

    analysis.add_fielding_event(
        match_no="IPL2023-045",
        innings=1,
        team="Mumbai Indians",
        player_name="Ishan Kishan",
        ballcount=1,
        position="Wicketkeeper",
        description="Missed stumping opportunity",
        pick="fumble",
        throw="missed stumping",
        runs=-2,
        overcount=10,
        venue="Wankhede Stadium"
    )

    analysis.add_fielding_event(
        match_no="IPL2023-045",
        innings=1,
        team="Mumbai Indians",
        player_name="Ishan Kishan",
        ballcount=4,
        position="Wicketkeeper",
        description="Good catch behind wicket",
        pick="catch",
        throw="",
        runs=1,
        overcount=13,
        venue="Wankhede Stadium"
    )

    # Calculate and display performance scores
    performance_scores = analysis.calculate_performance_score()
    print("\nPerformance Scores for all players:")
    print(performance_scores[['Player_Name', 'Team', 'Performance_Score']])

    # Generate player summaries
    print("\nDetailed player summaries:")
    analysis.player_summary()

    # Team analysis
    analysis.team_analysis(team_name="Mumbai Indians")

    # Position analysis
    analysis.position_analysis()

    # Export data
    analysis.export_data("fielding_sample_data.csv")

    # Visualize performance
    analysis.visualize_player_performance()

    print("\nDemo completed. Data exported to 'fielding_sample_data.csv'")


# Main function for interactive use
def main():
    """Interactive main function for Cricket Fielding Analysis System."""
    print("=" * 50)
    print("CRICKET FIELDING ANALYSIS SYSTEM")
    print("=" * 50)

    analysis = CricketFieldingAnalysis()

    while True:
        print("\nOptions:")
        print("1. Add fielding event")
        print("2. Calculate performance scores")
        print("3. View player summary")
        print("4. Team analysis")
        print("5. Position analysis")
        print("6. Visualize performance")
        print("7. Export data")
        print("8. Import data")
        print("9. Run demo with sample data")
        print("0. Exit")

        choice = input("\nEnter your choice (0-9): ")

        if choice == '1':
            # Add fielding event
            match_no = input("Match Number: ")
            innings = int(input("Innings (1/2): "))
            team = input("Team Name: ")
            player_name = input("Player Name: ")
            ballcount = int(input("Ball Count (1-6): "))
            position = input("Fielding Position: ")
            description = input("Short Description: ")

            print("\nPick options: clean pick, good throw, fumble, bad throw, catch, drop catch")
            pick = input("Pick: ").lower()

            print("\nThrow options: run out, missed stumping, missed run out, stumping (or leave blank)")
            throw = input("Throw: ").lower()

            runs = int(input("Runs Saved (+) or Conceded (-): "))
            overcount = int(input("Over Number: "))
            venue = input("Venue: ")

            analysis.add_fielding_event(
                match_no, innings, team, player_name, ballcount,
                position, description, pick, throw, runs, overcount, venue
            )

        elif choice == '2':
            # Calculate performance scores
            player = input("Player Name (leave blank for all): ")
            match = input("Match Number (leave blank for all): ")

            player = player if player else None
            match = match if match else None

            scores = analysis.calculate_performance_score(player, match)
            if not scores.empty:
                print("\nPerformance Scores:")
                print(scores[['Player_Name', 'Team', 'Performance_Score']])

        elif choice == '3':
            # View player summary
            player = input("Player Name (leave blank for all): ")
            match = input("Match Number (leave blank for all): ")

            player = player if player else None
            match = match if match else None

            analysis.player_summary(player, match)

        elif choice == '4':
            # Team analysis
            team = input("Team Name (leave blank for all): ")
            match = input("Match Number (leave blank for all): ")

            team = team if team else None
            match = match if match else None

            analysis.team_analysis(team, match)

        elif choice == '5':
            # Position analysis
            match = input("Match Number (leave blank for all): ")
            match = match if match else None

            analysis.position_analysis(match)

        elif choice == '6':
            # Visualize performance
            match = input("Match Number (leave blank for all): ")
            top_n = input("Number of top players to show (default 5): ")

            match = match if match else None
            top_n = int(top_n) if top_n.isdigit() else 5

            analysis.visualize_player_performance(match, top_n)

        elif choice == '7':
            # Export data
            filename = input("Filename (default: cricket_fielding_data.csv): ")
            filename = filename if filename else "cricket_fielding_data.csv"

            analysis.export_data(filename)

        elif choice == '8':
            # Import data
            filename = input("Filename to import: ")
            analysis.import_data(filename)

        elif choice == '9':
            # Run demo
            demo_with_sample_data()

        elif choice == '0':
            # Exit
            print("Thank you for using Cricket Fielding Analysis System!")
            break

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()

CRICKET FIELDING ANALYSIS SYSTEM

Options:
1. Add fielding event
2. Calculate performance scores
3. View player summary
4. Team analysis
5. Position analysis
6. Visualize performance
7. Export data
8. Import data
9. Run demo with sample data
0. Exit

Enter your choice (0-9): 6
Match Number (leave blank for all): IPL2023-045
Number of top players to show (default 5): 5
No fielding data available.

Options:
1. Add fielding event
2. Calculate performance scores
3. View player summary
4. Team analysis
5. Position analysis
6. Visualize performance
7. Export data
8. Import data
9. Run demo with sample data
0. Exit

Enter your choice (0-9): 6
Match Number (leave blank for all): IPL2023-045
Number of top players to show (default 5): 3
No fielding data available.

Options:
1. Add fielding event
2. Calculate performance scores
3. View player summary
4. Team analysis
5. Position analysis
6. Visualize performance
7. Export data
8. Import data
9. Run demo with sample data
0. Exit

Enter your choice